In [4]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

In [8]:
# 1，首先规定批量训练大小为2
batch_size = 2

In [10]:
# 2，初始化source和target词向量的长度，并转化为torch.int32格式）
# torch.randint(2, 4, (batch_size,))表示生成2到4之间的整数，元素个数为batch_size，并以元祖形式(batch_size,)保存
# 加逗号，否则会识别成batch_size本身
# 并转化成torch.int32格式，因为默认是int64，会多占用一般内存。我们不需要这么高精度
src_len = torch.randint(2, 4, (batch_size,)).to(torch.int32)
tgt_len = torch.randint(2, 4, (batch_size,)).to(torch.int32)
print(src_len) # 打印输出结果检查
print(tgt_len) # 同上

# 也就是说，src_len表示的是batch_size个句子，每个句子的长度为其元素
# 例如tensor([2, 3])，就是有两个句子，第一个句子长度为2，第二个句子长度为3

tensor([2, 3], dtype=torch.int32)
tensor([3, 2], dtype=torch.int32)


In [12]:
# 3，根据src_len和tgt_len的长度生成对应的序列
# 其中，[torch.randint(1, 8, (l,))的2和10表示最小单词量为1，最大单词量为8
# 用for循环遍历src_len的元素l，该元素作为初始化src_seq的大小形状
# 例如，src_len为tensor([2,4]),则src_seq的第一个元素就是tensor(a, b, 2)
# 最后用列表存储
src_seq = [torch.randint(1, 8, (l,)) for l in src_len]
tgt_seq = [torch.randint(1, 8, (l,)) for l in tgt_len]
print(src_seq)
print(tgt_seq)

# 例如src_seq = [torch.randint(1, 10, (l,)) for l in src_len]
# 由于src_len = tensor([2, 3])
# 因此for l in src_len循环的就是每个元素的具体值
# 对于第一个元素2，则生成一个从1到8随机取两个数的向量tensor([6, 7])
# 对于第二个元素3，则生成一个从1到8随机取三个数的向量tensor([4, 8, 8])


[tensor([6, 7]), tensor([4, 8, 8])]
[tensor([8, 5, 3]), tensor([4, 5])]


In [14]:
# 4，由于每个len的长度可能不一样，因此在这里要做一个padding填充
# 使得所有向量的形状大小一样，方便后续统一计算
# 因此引入统一变量，重新定义序列seq

# 设置单词表大小
max_num_src_words = 8
max_num_tgt_words = 8

# 定义每个padding后序列的最大长度
max_src_seg_len = 5
max_tgt_seg_len = 5

# 用F.pad填充。填充的输入是(torch.randint(1, max_num_src_words, (l,))
# 左边无填充所以是0
# 右边为自定义的序列最大长度减去遍历到的l
# ，因此对于，src_len的第一个元素是2，则最大长度-2=填充3个0
src_seq = [F.pad(torch.randint(1, max_num_src_words, (l,)), (0, max_src_seg_len - l)) for l in src_len]
tgt_seq = [F.pad(torch.randint(1, max_num_tgt_words, (l,)), (0, max_tgt_seg_len - l)) for l in tgt_len]

print(src_seq)
print(tgt_seq)

[tensor([4, 1, 0, 0, 0]), tensor([7, 6, 2, 0, 0])]
[tensor([1, 3, 7, 0, 0]), tensor([6, 5, 0, 0, 0])]


In [16]:
# 由于上述输出的src_seq和tgt_seq为两个分开的tensor向量，因此希望将向量改写成矩阵，以方便后续计算
# 继续改写src_seq和tgt_seq，使用stack函数
# 其中，tensor.stack(输入，dim）
# 输入一般为tensor向量或矩阵，dim为在哪个维度上连接，dim为0时则额外创建一个维度连接；dim为1时则行连接，等等
# 例如输入为一维向量时， dim=0会创建一个二维空间，在第二维度把向量放进去
# 输入为二维矩阵时，dim=0会在三维上放进去
# 以此类推

src_seq = [F.pad(torch.randint(1, max_num_src_words, (l,)), (0, max_src_seg_len - l)) for l in src_len]
src_seq = torch.stack(src_seq, 0)
print(src_seq)

tgt_seq = [F.pad(torch.randint(1, max_num_tgt_words, (l,)), (0, max_tgt_seg_len - l)) for l in tgt_len]
tgt_seq = torch.stack(tgt_seq, 0)
print(tgt_seq)

#最终得到tensor矩阵形式的src_seq和tgt_seq

tensor([[6, 1, 0, 0, 0],
        [3, 6, 2, 0, 0]])
tensor([[7, 3, 7, 0, 0],
        [6, 6, 0, 0, 0]])


In [20]:
# 5，构造embedding矩阵
# embedding本质就是一种特殊的独热编码
# embedding的具体用法参考外面的embedding学习的ipynb文件

model_dim = 8#根据transformer模型的超参来的，原文512，这里简化为8。这里对应原文的dmodel


# 用nn.Embedding初始化embedding矩阵，其大小为最大词库+1（因为padding的东西为0）乘model_dim
src_embedding_table = nn.Embedding(max_num_src_words+1, model_dim)

# 然后将src_seq作为输入传入embedding table里，得到关于每个单词（也就是每行）的编码信息
src_embedding = src_embedding_table(src_seq)

print(src_embedding_table.weight) # 打印src_seq中的每个单词（即原文的1、2、...、8以及pad得来的0共9个元素的关于model_dim的词embedding编码向量)
print(src_seq) # 上面的src_embedding_table.weight素材来源
print(src_embedding) # 用src_seq中涉及的每个单词看每个单词的词embedding编码向量

#同理，tgt也一样处理
tgt_embedding_table = nn.Embedding(max_num_tgt_words+1, model_dim)
tgt_embedding = src_embedding_table(tgt_seq)


print(tgt_embedding_table.weight)
print(tgt_seq)
print(tgt_embedding)

Parameter containing:
tensor([[ 0.3305, -0.7157, -1.5897,  1.1334, -0.2211, -0.7553,  1.1876, -0.9850],
        [-0.4791, -0.3004,  2.0029, -0.7475, -0.1174,  0.0927,  1.1969, -0.4423],
        [ 0.5184,  2.1956,  1.7249,  0.8338, -0.0674,  0.8243, -1.3604,  1.1947],
        [-2.2040, -0.1672,  0.9562,  1.5547,  1.4049,  0.3914,  0.3390, -2.5726],
        [ 1.6854,  0.8938, -0.6191, -0.4602, -0.8093,  0.8120,  1.0549,  0.4549],
        [ 1.4196,  0.7809,  0.4749,  1.4905,  0.4259, -0.4340,  0.6600, -1.2104],
        [ 1.0450, -0.7187, -0.1449,  0.8069,  0.0235, -0.2626, -0.1451,  0.4248],
        [-1.1366, -2.0651,  1.4856,  0.2466,  0.3595,  1.2130, -0.8386,  1.1096],
        [ 0.3650,  0.0262, -0.0271, -0.0484,  1.0287,  0.1781, -1.7947, -0.8245]],
       requires_grad=True)
tensor([[6, 1, 0, 0, 0],
        [3, 6, 2, 0, 0]])
tensor([[[ 1.0450, -0.7187, -0.1449,  0.8069,  0.0235, -0.2626, -0.1451,
           0.4248],
         [-0.4791, -0.3004,  2.0029, -0.7475, -0.1174,  0.0927,  1.1

In [46]:
# 定义序列最大长度
max_position_len = 5

# 构造position embedding,对应原文的pos
pos_mat = torch.arange(max_position_len).reshape((-1, 1))

# 构造i_mat，对应原文的i
i_mat = torch.pow(10000, torch.arange(0, 8, 2).reshape((1,-1))/model_dim)

# 构造一个pe_embedding_table，先初始化为0
pe_embedding_table = torch.zeros(max_position_len, model_dim)

#pe_embedding_table的偶数列为
pe_embedding_table[:, 0::2] = torch.sin(pos_mat / i_mat)

#pe_embedding_table的奇数列为
pe_embedding_table[:, 1::2] = torch.cos(pos_mat / i_mat)

#输出pe_embedding_table
print(pe_embedding_table)

pe_embedding = nn.Embedding(max_position_len, model_dim)
pe_embedding.weight = nn.Parameter(pe_embedding_table, requires_grad = False)

src_pos = torch.cat([torch.unsqueeze(torch.arange(max(src_len)), 0) for _ in src_len]).to(torch.int32)
tgt_pos = torch.cat([torch.unsqueeze(torch.arange(max(tgt_len)), 0) for _ in tgt_len]).to(torch.int32)

src_pe_embedding = pe_embedding(src_pos)
tgt_pe_embedding = pe_embedding(tgt_pos)

print(src_pe_embedding)
print(tgt_pe_embedding)


tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
          9.9995e-01,  1.0000e-03,  1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  1.9867e-01,  9.8007e-01,  1.9999e-02,
          9.9980e-01,  2.0000e-03,  1.0000e+00],
        [ 1.4112e-01, -9.8999e-01,  2.9552e-01,  9.5534e-01,  2.9995e-02,
          9.9955e-01,  3.0000e-03,  1.0000e+00],
        [-7.5680e-01, -6.5364e-01,  3.8942e-01,  9.2106e-01,  3.9989e-02,
          9.9920e-01,  4.0000e-03,  9.9999e-01]])
tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,
           1.0000e+00,  0.0000e+00,  1.0000e+00],
         [ 8.4147e-01,  5.4030e-01,  9.9833e-02,  9.9500e-01,  9.9998e-03,
           9.9995e-01,  1.0000e-03,  1.0000e+00],
         [ 9.0930e-01, -4.1615e-01,  1.9867e-01,  9.8007e-01,  1.9999e-02,
           9.9980e-01,  2.0000e-03,  1.0000e+00]],

       

In [ ]:
# 1 Attention(Q, K, V)
# 我们希望被mask的值放在softmax里变为负无穷，因此在softmax的结果中体现为0
# 因此我们需要构造一个矩阵，通过乘上这个矩阵，我们需要mask掉的元素变为负无穷，不需要mask的元素不变
# mask的size为(batch_size, max_src_len, max_src_len)，值为1和负无穷






